<a href="https://colab.research.google.com/github/Agnesing/UE/blob/main/Dotacje_UE_2014_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Umowy o dotacje z funduszy UE w Polsce w latach 2014-2020

Notebook przestawia wizualizację danych dotyczących  fudnuszy Unii Europejskiej w Polsce w perspektywie finansowej 2014-2020.

Dane pochodzą ze strony https://www.funduszeeuropejskie.gov.pl/

Dataset zawiera informacje o wszystkich umowach o dofinansowanie zawartych w Polsce wg stanu na dzień 31.03.2021 r.


In [1]:
import pandas as pd
import numpy as np

from urllib.request import urlretrieve #funkcja do importowania danych z internetu


In [3]:
# import danych
url="https://www.funduszeeuropejskie.gov.pl/media/99960/045_Umowy_pelna_lista_2021_03_31.xlsx"

urlretrieve(url, 'umowy.xlsx')


('umowy.xlsx', <http.client.HTTPMessage at 0x7f23cbe7add0>)

In [4]:
#wczytanie pobranego pliku do Pandas
file='umowy.xlsx'
data=pd.ExcelFile(file)

In [6]:
#teraz możemy zobaczyć nazwy arkuszy
print(data.sheet_names)

['Umowy pełna lista']


In [7]:
df=data.parse('Umowy pełna lista')

In [8]:
df.head()

,Realizacja programów na lata 2014-2020: Umowy według miejsc realizacji i innych kategorii (stan na 31 marca 2021 roku - na podstawie danych z Centralnego Systemu Teleinformatycznego SL2014) [Data wygenerowania raportu: 2021-04-01 09:24:03],Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20,Unnamed: 21,Unnamed: 22,Unnamed: 23,Unnamed: 24,Unnamed: 25,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34
0,Numer projektu,Tytuł projektu,Program operacyjny,Priorytet,Działanie,Poddziałanie,Fundusz,Miejsce realizacji - województwo,Miejsce realizacji - powiat,Miejsce realizacji - gmina,Wartość ogółem (PLN),Wydatki kwalifikowlne (PLN),Dofinansowanie (PLN),Dofinansowanie UE (PLN),Nazwa beneficjenta (wiodącego),Beneficjent NIP,Beneficjent kod pocztowy,Beneficjent miejscowosć,Beneficjent województwo,Beneficjent powiat,Beneficjent gmina,Beneficjent forma prawna,Wszyscy partnerzy projektu,Cel Tematyczny,Zakres interwencji (dominujący),Temat uzupełniający dla projektów EFS,Forma finansowania,Typ obszaru realizacji,Rodzaj działalności gospodarczej,Projekt realizowany w ramach terytorialnych me...,Data rozpoczęcia realizacji,Data zakończenia realizacji,Data podpisania umowy pierwotnej,Data podpisania aktualnej wersji umowy,Data utworzenia w SL 2014 aktualnej wersji umowy
1,POIR.01.01.01-00-0001/20,Opracowanie nowatorskiej technologii krojenia ...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,1.1.1. Badania przemysłowe i prace rozwojowe r...,EFRR,LUBELSKIE,radzyński,Radzyń Podlaski,2.49065e+07,2.49065e+07,9.9626e+06,9.9626e+06,SPÓŁDZIELCZA MLECZARNIA SPOMLEK,5380002604,21-300,Radzyń Podlaski,LUBELSKIE,radzyński,Radzyń Podlaski,spółdzielnie,SPÓŁDZIELCZA MLECZARNIA SPOMLEK,"01 Wzmacnianie badań naukowych, rozwoju techno...",002 Procesy badawcze i innowacyjne w dużych pr...,NaN,01 Dotacja bezzwrotna,07 Nie dotyczy,03 Produkcja artykułów spożywczych i napojów,NaN,2020-03-01 00:00:00,2021-12-31 00:00:00,2020-09-02 00:00:00,2020-09-02 00:00:00,2020-09-05 00:00:00
2,POIR.01.01.01-00-0002/17,Mobilna linia przetwarzania odpadów komunalnyc...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,1.1.1. Badania przemysłowe i prace rozwojowe r...,EFRR,PODLASKIE,NaN,NaN,49335889,49335889,2.3521e+07,2.3521e+07,PRONAR SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,5430200939,17-210,Narew,PODLASKIE,hajnowski,Narew,spółki z ograniczoną odpowiedzialnością - duże...,PRONAR SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,"01 Wzmacnianie badań naukowych, rozwoju techno...",002 Procesy badawcze i innowacyjne w dużych pr...,NaN,01 Dotacja bezzwrotna,07 Nie dotyczy,24 Inne niewyszczególnione usługi,NaN,2017-04-03 00:00:00,2020-06-30 00:00:00,2017-08-17 00:00:00,2020-12-08 00:00:00,2020-12-09 00:00:00
3,POIR.01.01.01-00-0002/20,Automatyzacja procesu produkcji wyrobów/elemen...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,1.1.1. Badania przemysłowe i prace rozwojowe r...,EFRR,ŁÓDZKIE,Łódź,NaN,7.11108e+06,7.11108e+06,2.86384e+06,2.86384e+06,&BDQUO;HSV POLSKA&RDQUO; SP. Z O.O.,7491833077,94-104,Łódź,ŁÓDZKIE,Łódź,Łódź,spółki z ograniczoną odpowiedzialnością - duże...,&BDQUO;HSV POLSKA&RDQUO; SP. Z O.O.,"01 Wzmacnianie badań naukowych, rozwoju techno...",002 Procesy badawcze i innowacyjne w dużych pr...,NaN,01 Dotacja bezzwrotna,07 Nie dotyczy,07 Pozostałe nieokreślone branże przemysłu wyt...,NaN,2020-03-03 00:00:00,2021-08-31 00:00:00,2020-09-02 00:00:00,2020-09-02 00:00:00,2020-09-02 00:00:00
4,POIR.01.01.01-00-0003/19,Platforama Business Intelligence firmy SESCOM ...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B

In [11]:
#Pierwszy wiersz jest zbędny, drugi wiersz zawiera nagłówki kolumn. Przekształćmy w ten sposób dataframe
#po usunięciu pierwszego wiersza, nagłówek znajdzie się w pierwszym wierszu o indeksie 0.
df=data.parse('Umowy pełna lista', header=0, skiprows=1) 
df.head()

,Numer projektu,Tytuł projektu,Program operacyjny,Priorytet,Działanie,Poddziałanie,Fundusz,Miejsce realizacji - województwo,Miejsce realizacji - powiat,Miejsce realizacji - gmina,Wartość ogółem (PLN),Wydatki kwalifikowlne (PLN),Dofinansowanie (PLN),Dofinansowanie UE (PLN),Nazwa beneficjenta (wiodącego),Beneficjent NIP,Beneficjent kod pocztowy,Beneficjent miejscowosć,Beneficjent województwo,Beneficjent powiat,Beneficjent gmina,Beneficjent forma prawna,Wszyscy partnerzy projektu,Cel Tematyczny,Zakres interwencji (dominujący),Temat uzupełniający dla projektów EFS,Forma finansowania,Typ obszaru realizacji,Rodzaj działalności gospodarczej,Projekt realizowany w ramach terytorialnych mechanizmów,Data rozpoczęcia realizacji,Data zakończenia realizacji,Data podpisania umowy pierwotnej,Data podpisania aktualnej wersji umowy,Data utworzenia w SL 2014 aktualnej wersji umowy
0,POIR.01.01.01-00-0001/20,Opracowanie nowatorskiej technologii krojenia ...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,1.1.1. Badania przemysłowe i prace rozwojowe r...,EFRR,LUBELSKIE,radzyński,Radzyń Podlaski,24906510.50,24906510.50,9962604.20,9962604.20,SPÓŁDZIELCZA MLECZARNIA SPOMLEK,5380002604,21-300,Radzyń Podlaski,LUBELSKIE,radzyński,Radzyń Podlaski,spółdzielnie,SPÓŁDZIELCZA MLECZARNIA SPOMLEK,"01 Wzmacnianie badań naukowych, rozwoju techno...",002 Procesy badawcze i innowacyjne w dużych pr...,NaN,01 Dotacja bezzwrotna,07 Nie dotyczy,03 Produkcja artykułów spożywczych i napojów,NaN,2020-03-01,2021-12-31,2020-09-02,2020-09-02,2020-09-05
1,POIR.01.01.01-00-0002/17,Mobilna linia przetwarzania odpadów komunalnyc...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,1.1.1. Badania przemysłowe i prace rozwojowe r...,EFRR,PODLASKIE,NaN,NaN,49335889.00,49335889.00,23520969.39,23520969.39,PRONAR SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,5430200939,17-210,Narew,PODLASKIE,hajnowski,Narew,spółki z ograniczoną odpowiedzialnością - duże...,PRONAR SPÓŁKA Z OGRANICZONĄ ODPOWIEDZIALNOŚCIĄ,"01 Wzmacnianie badań naukowych, rozwoju techno...",002 Procesy badawcze i innowacyjne w dużych pr...,NaN,01 Dotacja bezzwrotna,07 Nie dotyczy,24 Inne niewyszczególnione usługi,NaN,2017-04-03,2020-06-30,2017-08-17,2020-12-08,2020-12-09
2,POIR.01.01.01-00-0002/20,Automatyzacja procesu produkcji wyrobów/elemen...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,1.1.1. Badania przemysłowe i prace rozwojowe r...,EFRR,ŁÓDZKIE,Łódź,NaN,7111078.77,7111078.77,2863843.28,2863843.28,&BDQUO;HSV POLSKA&RDQUO; SP. Z O.O.,7491833077,94-104,Łódź,ŁÓDZKIE,Łódź,Łódź,spółki z ograniczoną odpowiedzialnością - duże...,&BDQUO;HSV POLSKA&RDQUO; SP. Z O.O.,"01 Wzmacnianie badań naukowych, rozwoju techno...",002 Procesy badawcze i innowacyjne w dużych pr...,NaN,01 Dotacja bezzwrotna,07 Nie dotyczy,07 Pozostałe nieokreślone branże przemysłu wyt...,NaN,2020-03-03,2021-08-31,2020-09-02,2020-09-02,2020-09-02
3,POIR.01.01.01-00-0003/19,Platforama Business Intelligence firmy SESCOM ...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,1.1.1. Badania przemysłowe i prace rozwojowe r...,EFRR,POMORSKIE,Gdańsk,Gdańsk,3195275.00,3195275.00,2199575.00,2199575.00,SESCOM SPÓŁKA AKCYJNA,9571006288,80-244,Gdańsk,POMORSKIE,Gdańsk,Gdańsk,spółki akcyjne - średnie przedsiębiorstwo,SESCOM SPÓŁKA AKCYJNA,"01 Wzmacnianie badań naukowych, rozwoju techno...",064 Procesy badawcze i innowacyjne w MŚP (w ty...,NaN,01 Dotacja bezzwrotna,07 Nie dotyczy,"13 Działania informacyjno-komunikacyjne, w tym...",NaN,2019-10-01,2021-09-30,2019-09-24,2019-09-24,2019-09-25
4,POIR.01.01.01-00-0004/15,Opracowanie technologii produkcji jonowych far...,Program Operacyjny Inteligentny Rozwój,1. Wsparcie prowadzenia prac B+R przez przedsi...,1.1. Projekty B+R przedsiębiorstw,

In [13]:
# sprawdźmy jak kończy się tabela
df.tail(10)

,Numer projektu,Tytuł projektu,Program operacyjny,Priorytet,Działanie,Poddziałanie,Fundusz,Miejsce realizacji - województwo,Miejsce realizacji - powiat,Miejsce realizacji - gmina,Wartość ogółem (PLN),Wydatki kwalifikowlne (PLN),Dofinansowanie (PLN),Dofinansowanie UE (PLN),Nazwa beneficjenta (wiodącego),Beneficjent NIP,Beneficjent kod pocztowy,Beneficjent miejscowosć,Beneficjent województwo,Beneficjent powiat,Beneficjent gmina,Beneficjent forma prawna,Wszyscy partnerzy projektu,Cel Tematyczny,Zakres interwencji (dominujący),Temat uzupełniający dla projektów EFS,Forma finansowania,Typ obszaru realizacji,Rodzaj działalności gospodarczej,Projekt realizowany w ramach terytorialnych mechanizmów,Data rozpoczęcia realizacji,Data zakończenia realizacji,Data podpisania umowy pierwotnej,Data podpisania aktualnej wersji umowy,Data utworzenia w SL 2014 aktualnej wersji umowy
200580,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200581,Uwagi:,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200582,Zaprojektowany algorytm przeliczania oparty je...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200583,1. Ponieważ każdy z partnerów Projektu może mi...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200584,2. Województwa mają udziały proporcjonalnie w ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200585,"3. Powiaty mają udziały proporcjonalnie, ale w...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200586,"4. Gminy mają udziały proporcjonalnie, ale w r...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200587,5. Kwota w ramach Projektu dla danego miejsca ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200588,"W przypadku projektów partnerskich, wszystkie ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT
200589,W związku z podziałem projektów na miejsca rea...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaT


In [19]:
# Ostatnie wiersze zawierają uwagi metodologiczne dotyczące określenia miejsca realizacji projektu. 
# Znajdują się w pierwszej kolumnie. Zobaczmy je w całości

print(df.iloc[200581:200589, 0].values)

['Uwagi: '
 'Zaprojektowany algorytm przeliczania oparty jest na miejscach realizacji zarejestrowanych we wnioskach o dofinansowanie i umowach o dofinansowanie - działa tylko przy poprawnie zdefiniowanych w SL 2014 miejscach realizacji. W związku z tym, że nigdzie w SL 2014 nie zostało zaprojektowane sztywne przypisanie konkretnych wartości do konkretnych miejsc realizacji, dane prezentowane w raportach należy traktować jako przybliżone, wyliczane w jeden ustalony sposób: '
 '1. Ponieważ każdy z partnerów Projektu może mieć własne miejsca realizacji, kwoty najpierw dzieli się na poszczególnych partnerów według tego, co jest w SL 2014. Następnie dla każdego z partnerów obliczoną kwotę dzieli się na jego miejsca realizacji. '
 '2. Województwa mają udziały proporcjonalnie w ramach Projektu: udział Województwa = kwota Projektu dzielona przez liczbę Województw. '
 '3. Powiaty mają udziały proporcjonalnie, ale w ramach swojego Województwa: udział Powiatu = kwota udziału Województwa dzielona 

In [40]:
#Analizę wykonamy na poziomie województw. Badamy skuteczność zawierania umów na poziomie województw, 
#stąd do analizy przyjmiemy siedzibę beneficjenta, a nie miejsce ralizacji proejtku.
#Potrzebujemy następujących kolumn: 'Beneficjent województwo', 'Dofinansowanie UE (PLN)', 'Data podpisania umowy pierwotnej'.

df1=df[['Beneficjent województwo', 'Dofinansowanie UE (PLN)', 'Data podpisania umowy pierwotnej']]

#zobaczy kikla przykładowych rekordów
df1.head(5)

,Beneficjent województwo,Dofinansowanie UE (PLN),Data podpisania umowy pierwotnej
0,LUBELSKIE,9962604.20,2020-09-02
1,PODLASKIE,23520969.39,2017-08-17
2,ŁÓDZKIE,2863843.28,2020-09-02
3,POMORSKIE,2199575.00,2019-09-24
4,OPOLSKIE,1676328.13,2015-10-26


In [39]:
#zapiszmy dane dotyczące dofinansowania w tys. PLN
#df1['Dofinansowanie tys pln']=df1['Dofinansowanie UE (PLN)']/1000
#zaokrąglenie do liczb całkowitych
#df1['Dofinansowanie tys pln']=df1['Dofinansowanie tys pln'].round(0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [48]:
#Z uwagi na duży zakres dat dziennych, przeskaluję je na miesiące, wykazując sumę dofinansowania dla poszczególnych województw w kolejnych miesiącach.
dotacje=df1.groupby('Beneficjent województwo').resample('M', on = 'Data podpisania umowy pierwotnej')['Dofinansowanie UE (PLN)'].sum()
dotacje

Beneficjent województwo  Data podpisania umowy pierwotnej
DOLNOŚLĄSKIE             2015-05-31                          8.392782e+07
                         2015-06-30                          7.578065e+07
                         2015-07-31                          7.660618e+07
                         2015-08-31                          4.211999e+07
                         2015-09-30                          1.325597e+08
                                                                 ...     
ŚWIĘTOKRZYSKIE           2020-10-31                          5.244336e+05
                         2020-11-30                          0.000000e+00
                         2020-12-31                          2.526609e+06
                         2021-01-31                          8.078400e+05
                         2021-02-28                          1.722599e+06
Name: Dofinansowanie UE (PLN), Length: 1166, dtype: float64

In [59]:
#zapisujemy wynik do dataframe
df2=pd.DataFrame(dotacje)
df2.head()

Dofinansowanie UE (PLN)
Beneficjent województwo Data podpisania umowy pierwotnej                         
DOLNOŚLĄSKIE            2015-05-31                                   8.392782e+07
                        2015-06-30                                   7.578065e+07
                        2015-07-31                                   7.660618e+07
                        2015-08-31                                   4.211999e+07
                        2015-09-30                                   1.325597e+08

In [60]:
# Otrzymaliśmy tabelę z multiindeksem. Należy go zresetować
df2=df2.reset_index()
df2

,Beneficjent województwo,Data podpisania umowy pierwotnej,Dofinansowanie UE (PLN)
0,DOLNOŚLĄSKIE,2015-05-31,8.392782e+07
1,DOLNOŚLĄSKIE,2015-06-30,7.578065e+07
2,DOLNOŚLĄSKIE,2015-07-31,7.660618e+07
3,DOLNOŚLĄSKIE,2015-08-31,4.211999e+07
4,DOLNOŚLĄSKIE,2015-09-30,1.325597e+08
...,...,...,...
1161,ŚWIĘTOKRZYSKIE,2020-10-31,5.244336e+05
1162,ŚWIĘTOKRZYSKIE,2020-11-30,0.000000e+00
1163,ŚWIĘTOKRZYSKIE,2020-12-31,2.526609e+06
1164,ŚWIĘTOKRZYSKIE,2021-01-31,8.078400e+05


In [61]:
#Przekształcamy tabelę aby w wierszach były nazwy województw a w kolumnach poszczególne miesiące
df3=df2.pivot(index='Beneficjent województwo', columns='Data podpisania umowy pierwotnej', values='Dofinansowanie UE (PLN)')
df3

Data podpisania umowy pierwotnej,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
Beneficjent województwo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DOLNOŚLĄSKIE,NaN,NaN,NaN,NaN,NaN,8.392782e+07,7.578065e+07,7.660618e+07,4.211999e+07,1.325597e+08,1.987041e+08,2.236507e+08,2.814020e+08,5.474592e+07,1.381204e+08,3.531052e+08,3.388852e+07,2.394189e+08,3.229700e+05,53994210.03,7.795179e+07,1.262962e+09,1.561554e+09,2.039537e+08,2.511963e+08,2.826012e+08,1.381383e+08,3.076323e+08,9.688561e+08,1.009126e+09,5.064630e+09,5.357393e+08,1.802066e+07,4.655034e+06,9.500013e+06,4.474159e+07,2.952387e+07,3.593411e+06,0.000000e+00,3.730247e+06,2.196880e+07,4.313439e+06,3.039196e+06,5.689668e+06,1.455584e+07,1.408139e+07,1.916645e+07,4.820396e+06,4.307158e+06,2.588825e+06,7.902801e+06,2.174623e+07,3.609084e+06,2.399368e+06,2.782839e+07,1.436071e+06,0.000000e+00,2.554905e+06,3.579769e+06,4.119257e+06,3.016315e+06,0.000000e+00,5.081999e+05,0.000000e+00,0.000000e+00,0.000000e+00,6.904847e+06,3.954987e+06,8.801251e+05,7.873217e+06,7.355587e+06,7.080795e+06,5.084956e+06,1.112435e+06,1.995608e+07,7.256993e+06
KUJAWSKO-POMORSKIE,NaN,NaN,NaN,NaN,NaN,1.171917e+07,7.521109e+07,1.433124e+06,1.893816e+07,3.201881e+07,8.255317e+07,8.719112e+07,1.880686e+08,3.245439e+07,4.555235e+07,3.809631e+08,4.084056e+07,4.426745e+08,0.000000e+00,50477048.18,1.184790e+07,8.683857e+08,1.184365e+09,5.386252e+06,2.428215e+08,2.441316e+08,1.327805e+08,1.603305e+08,1.600020e+08,4.076439e+04,0.000000e+00,5.452900e+09,1.653789e+09,0.000000e+00,1.069661e+07,2.217413e+06,8.001626e+06,2.366451e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.591160e+06,7.549855e+05,0.000000e+00,5.753277e+06,0.000000e+00,7.106224e+06,2.502508e+06,0.000000e+00,2.299088e+07,6.489262e+06,2.421932e+06,1.613867e+07,2.545751e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.090000e+05,0.000000e+00,2.923130e+06,1.764281e+07,1.917146e+07,3.319990e+06,1.211905e+06,0.000000e+00,1.411623e+06,0.000000e+00,6.238251e+06,2.746761e+06,1.406005e+06,1.429135e+07,7.916475e+05,4.375865e+06,2.465753e+06
LUBELSKIE,NaN,NaN,NaN,NaN,NaN,1.631861e+07,5.851541e+07,5.556843e+06,4.178929e+07,1.643306e+07,2.355843e+07,1.124698e+08,8.577586e+07,2.136808e+07,1.219349e+07,4.597729e+08,1.825488e+07,2.960811e+08,4.110400e+05,0.00,1.872606e+07,8.354670e+08,5.923305e+08,6.901768e+06,1.677562e+09,2.485394e+08,2.398477e+08,2.656193e+08,1.540757e+08,0.000000e+00,1.399048e+07,1.060135e+07,1.454053e+07,7.495108e+06,1.270431e+07,1.015938e+07,7.005467e+09,1.630208e+09,1.093090e+05,3.778683e+06,5.601717e+06,1.033263e+07,8.785949e+06,1.313085e+07,1.882097e+07,1.105154e+07,9.041116e+06,1.086341e+06,2.334783e+07,1.380782e+07,9.565680e+06,2.094293e+07,1.495717e+06,5.455124e+06,1.146633e+07,1.985921e+07,0.000000e+00,2.867939e+06,2.690442e+07,1.694644e+06,0.000000e+00,8.230656e+05,0.000000e+00,1.197106e+07,0.000000e+00,5.324709e+05,1.214555e+07,1.607011e+06,0.000000e+00,4.246990e+06,7.970617e+06,8.190733e+06,4.949072e+06,3.149904e+06,1.486291e+06,6.403360e+06
LUBUSKIE,NaN,NaN,NaN,NaN,NaN,3.418706e+07,0.000000e+00,3.549128e+07,1.639207e+07,2.708903e+07,2.053042e+07,2.070763e+07,1.554652e+07,5.855674e+06,8.186287e+06,2.965943e+0

In [62]:
#warości NaN oznaczają, że w danym miesiącu nie zawarto żadnych umów o dofinansowanie.
#Wypełniam je zerami
df3=df3.fillna(0)
df3
#teraz wyznaczamy sumy narastające dla każdego województwa. W kolejnych mieiśacach będziemy wykazywać sumę wartości dotacji od początku perspektywy.

Data podpisania umowy pierwotnej,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
Beneficjent województwo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DOLNOŚLĄSKIE,0.00,0.0,0.00,0.0,0.0,8.392782e+07,7.578065e+07,7.660618e+07,4.211999e+07,1.325597e+08,1.987041e+08,2.236507e+08,2.814020e+08,5.474592e+07,1.381204e+08,3.531052e+08,3.388852e+07,2.394189e+08,3.229700e+05,53994210.03,7.795179e+07,1.262962e+09,1.561554e+09,2.039537e+08,2.511963e+08,2.826012e+08,1.381383e+08,3.076323e+08,9.688561e+08,1.009126e+09,5.064630e+09,5.357393e+08,1.802066e+07,4.655034e+06,9.500013e+06,4.474159e+07,2.952387e+07,3.593411e+06,0.000000e+00,3.730247e+06,2.196880e+07,4.313439e+06,3.039196e+06,5.689668e+06,1.455584e+07,1.408139e+07,1.916645e+07,4.820396e+06,4.307158e+06,2.588825e+06,7.902801e+06,2.174623e+07,3.609084e+06,2.399368e+06,2.782839e+07,1.436071e+06,0.000000e+00,2.554905e+06,3.579769e+06,4.119257e+06,3.016315e+06,0.000000e+00,5.081999e+05,0.000000e+00,0.000000e+00,0.000000e+00,6.904847e+06,3.954987e+06,8.801251e+05,7.873217e+06,7.355587e+06,7.080795e+06,5.084956e+06,1.112435e+06,1.995608e+07,7.256993e+06
KUJAWSKO-POMORSKIE,0.00,0.0,0.00,0.0,0.0,1.171917e+07,7.521109e+07,1.433124e+06,1.893816e+07,3.201881e+07,8.255317e+07,8.719112e+07,1.880686e+08,3.245439e+07,4.555235e+07,3.809631e+08,4.084056e+07,4.426745e+08,0.000000e+00,50477048.18,1.184790e+07,8.683857e+08,1.184365e+09,5.386252e+06,2.428215e+08,2.441316e+08,1.327805e+08,1.603305e+08,1.600020e+08,4.076439e+04,0.000000e+00,5.452900e+09,1.653789e+09,0.000000e+00,1.069661e+07,2.217413e+06,8.001626e+06,2.366451e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.591160e+06,7.549855e+05,0.000000e+00,5.753277e+06,0.000000e+00,7.106224e+06,2.502508e+06,0.000000e+00,2.299088e+07,6.489262e+06,2.421932e+06,1.613867e+07,2.545751e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.090000e+05,0.000000e+00,2.923130e+06,1.764281e+07,1.917146e+07,3.319990e+06,1.211905e+06,0.000000e+00,1.411623e+06,0.000000e+00,6.238251e+06,2.746761e+06,1.406005e+06,1.429135e+07,7.916475e+05,4.375865e+06,2.465753e+06
LUBELSKIE,0.00,0.0,0.00,0.0,0.0,1.631861e+07,5.851541e+07,5.556843e+06,4.178929e+07,1.643306e+07,2.355843e+07,1.124698e+08,8.577586e+07,2.136808e+07,1.219349e+07,4.597729e+08,1.825488e+07,2.960811e+08,4.110400e+05,0.00,1.872606e+07,8.354670e+08,5.923305e+08,6.901768e+06,1.677562e+09,2.485394e+08,2.398477e+08,2.656193e+08,1.540757e+08,0.000000e+00,1.399048e+07,1.060135e+07,1.454053e+07,7.495108e+06,1.270431e+07,1.015938e+07,7.005467e+09,1.630208e+09,1.093090e+05,3.778683e+06,5.601717e+06,1.033263e+07,8.785949e+06,1.313085e+07,1.882097e+07,1.105154e+07,9.041116e+06,1.086341e+06,2.334783e+07,1.380782e+07,9.565680e+06,2.094293e+07,1.495717e+06,5.455124e+06,1.146633e+07,1.985921e+07,0.000000e+00,2.867939e+06,2.690442e+07,1.694644e+06,0.000000e+00,8.230656e+05,0.000000e+00,1.197106e+07,0.000000e+00,5.324709e+05,1.214555e+07,1.607011e+06,0.000000e+00,4.246990e+06,7.970617e+06,8.190733e+06,4.949072e+06,3.149904e+06,1.486291e+06,6.403360e+06
LUBUSKIE,0.00,0.0,0.00,0.0,0.0,3.418706e+07,0.000000e+00,3.549128e+07,1.639207e+07,2.708903e+07,2.053042e+07,2.070763e+07,1.554652e+07,5.855674e+06,8.186287e+06,2.9

In [63]:
# wiersz "brak danych" zawiera informacje dotyczące umów dla których nie podano siedziby beneficjenta.
# Nie mamy możliwości, żeby je uzupełnić. Zatem usuwamy ten wiersz.
df3=df3.drop(labels='brak danych', axis=0)
df3

Data podpisania umowy pierwotnej,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
Beneficjent województwo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DOLNOŚLĄSKIE,0.00,0.0,0.00,0.0,0.0,8.392782e+07,7.578065e+07,7.660618e+07,4.211999e+07,1.325597e+08,1.987041e+08,2.236507e+08,2.814020e+08,5.474592e+07,1.381204e+08,3.531052e+08,3.388852e+07,2.394189e+08,3.229700e+05,53994210.03,7.795179e+07,1.262962e+09,1.561554e+09,2.039537e+08,2.511963e+08,2.826012e+08,1.381383e+08,3.076323e+08,9.688561e+08,1.009126e+09,5.064630e+09,5.357393e+08,1.802066e+07,4.655034e+06,9.500013e+06,4.474159e+07,2.952387e+07,3.593411e+06,0.000000e+00,3.730247e+06,2.196880e+07,4.313439e+06,3.039196e+06,5.689668e+06,1.455584e+07,1.408139e+07,1.916645e+07,4.820396e+06,4.307158e+06,2.588825e+06,7.902801e+06,2.174623e+07,3.609084e+06,2.399368e+06,2.782839e+07,1.436071e+06,0.000000e+00,2.554905e+06,3.579769e+06,4.119257e+06,3.016315e+06,0.000000e+00,5.081999e+05,0.000000e+00,0.000000e+00,0.000000e+00,6.904847e+06,3.954987e+06,8.801251e+05,7.873217e+06,7.355587e+06,7.080795e+06,5.084956e+06,1.112435e+06,1.995608e+07,7.256993e+06
KUJAWSKO-POMORSKIE,0.00,0.0,0.00,0.0,0.0,1.171917e+07,7.521109e+07,1.433124e+06,1.893816e+07,3.201881e+07,8.255317e+07,8.719112e+07,1.880686e+08,3.245439e+07,4.555235e+07,3.809631e+08,4.084056e+07,4.426745e+08,0.000000e+00,50477048.18,1.184790e+07,8.683857e+08,1.184365e+09,5.386252e+06,2.428215e+08,2.441316e+08,1.327805e+08,1.603305e+08,1.600020e+08,4.076439e+04,0.000000e+00,5.452900e+09,1.653789e+09,0.000000e+00,1.069661e+07,2.217413e+06,8.001626e+06,2.366451e+06,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.591160e+06,7.549855e+05,0.000000e+00,5.753277e+06,0.000000e+00,7.106224e+06,2.502508e+06,0.000000e+00,2.299088e+07,6.489262e+06,2.421932e+06,1.613867e+07,2.545751e+07,0.000000e+00,0.000000e+00,0.000000e+00,8.090000e+05,0.000000e+00,2.923130e+06,1.764281e+07,1.917146e+07,3.319990e+06,1.211905e+06,0.000000e+00,1.411623e+06,0.000000e+00,6.238251e+06,2.746761e+06,1.406005e+06,1.429135e+07,7.916475e+05,4.375865e+06,2.465753e+06
LUBELSKIE,0.00,0.0,0.00,0.0,0.0,1.631861e+07,5.851541e+07,5.556843e+06,4.178929e+07,1.643306e+07,2.355843e+07,1.124698e+08,8.577586e+07,2.136808e+07,1.219349e+07,4.597729e+08,1.825488e+07,2.960811e+08,4.110400e+05,0.00,1.872606e+07,8.354670e+08,5.923305e+08,6.901768e+06,1.677562e+09,2.485394e+08,2.398477e+08,2.656193e+08,1.540757e+08,0.000000e+00,1.399048e+07,1.060135e+07,1.454053e+07,7.495108e+06,1.270431e+07,1.015938e+07,7.005467e+09,1.630208e+09,1.093090e+05,3.778683e+06,5.601717e+06,1.033263e+07,8.785949e+06,1.313085e+07,1.882097e+07,1.105154e+07,9.041116e+06,1.086341e+06,2.334783e+07,1.380782e+07,9.565680e+06,2.094293e+07,1.495717e+06,5.455124e+06,1.146633e+07,1.985921e+07,0.000000e+00,2.867939e+06,2.690442e+07,1.694644e+06,0.000000e+00,8.230656e+05,0.000000e+00,1.197106e+07,0.000000e+00,5.324709e+05,1.214555e+07,1.607011e+06,0.000000e+00,4.246990e+06,7.970617e+06,8.190733e+06,4.949072e+06,3.149904e+06,1.486291e+06,6.403360e+06
LUBUSKIE,0.00,0.0,0.00,0.0,0.0,3.418706e+07,0.000000e+00,3.549128e+07,1.639207e+07,2.708903e+07,2.053042e+07,2.070763e+07,1.554652e+07,5.855674e+06,8.186287e+06,2.9

In [65]:
#teraz wyznaczamy sumy narastające dla każdego województwa. W kolejnych miesiącach będziemy wykazywać sumę wartości dotacji od początku perspektywy.
df4=df3.cumsum(axis=1)
df4

Data podpisania umowy pierwotnej,2014-12-31,2015-01-31,2015-02-28,2015-03-31,2015-04-30,2015-05-31,2015-06-30,2015-07-31,2015-08-31,2015-09-30,2015-10-31,2015-11-30,2015-12-31,2016-01-31,2016-02-29,2016-03-31,2016-04-30,2016-05-31,2016-06-30,2016-07-31,2016-08-31,2016-09-30,2016-10-31,2016-11-30,2016-12-31,2017-01-31,2017-02-28,2017-03-31,2017-04-30,2017-05-31,2017-06-30,2017-07-31,2017-08-31,2017-09-30,2017-10-31,2017-11-30,2017-12-31,2018-01-31,2018-02-28,2018-03-31,2018-04-30,2018-05-31,2018-06-30,2018-07-31,2018-08-31,2018-09-30,2018-10-31,2018-11-30,2018-12-31,2019-01-31,2019-02-28,2019-03-31,2019-04-30,2019-05-31,2019-06-30,2019-07-31,2019-08-31,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30,2020-07-31,2020-08-31,2020-09-30,2020-10-31,2020-11-30,2020-12-31,2021-01-31,2021-02-28,2021-03-31
Beneficjent województwo,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
DOLNOŚLĄSKIE,0.00,0.00,0.00,0.00,0.00,8.392782e+07,1.597085e+08,2.363146e+08,2.784346e+08,4.109943e+08,6.096984e+08,8.333492e+08,1.114751e+09,1.169497e+09,1.307617e+09,1.660723e+09,1.694611e+09,1.934030e+09,1.934353e+09,1.988347e+09,2.066299e+09,3.329261e+09,4.890816e+09,5.094769e+09,5.345966e+09,5.628567e+09,5.766705e+09,6.074337e+09,7.043193e+09,8.052320e+09,1.311695e+10,1.365269e+10,1.367071e+10,1.367536e+10,1.368486e+10,1.372961e+10,1.375913e+10,1.376272e+10,1.376272e+10,1.376645e+10,1.378842e+10,1.379274e+10,1.379578e+10,1.380146e+10,1.381602e+10,1.383010e+10,1.384927e+10,1.385409e+10,1.385840e+10,1.386098e+10,1.386889e+10,1.389063e+10,1.389424e+10,1.389664e+10,1.392447e+10,1.392591e+10,1.392591e+10,1.392846e+10,1.393204e+10,1.393616e+10,1.393918e+10,1.393918e+10,1.393969e+10,1.393969e+10,1.393969e+10,1.393969e+10,1.394659e+10,1.395055e+10,1.395143e+10,1.395930e+10,1.396665e+10,1.397373e+10,1.397882e+10,1.397993e+10,1.399989e+10,1.400715e+10
KUJAWSKO-POMORSKIE,0.00,0.00,0.00,0.00,0.00,1.171917e+07,8.693026e+07,8.836338e+07,1.073015e+08,1.393203e+08,2.218735e+08,3.090646e+08,4.971333e+08,5.295877e+08,5.751400e+08,9.561030e+08,9.969436e+08,1.439618e+09,1.439618e+09,1.490095e+09,1.501943e+09,2.370329e+09,3.554694e+09,3.560080e+09,3.802901e+09,4.047033e+09,4.179814e+09,4.340144e+09,4.500146e+09,4.500187e+09,4.500187e+09,9.953086e+09,1.160688e+10,1.160688e+10,1.161757e+10,1.161979e+10,1.162779e+10,1.163016e+10,1.163016e+10,1.163016e+10,1.163016e+10,1.163016e+10,1.163016e+10,1.163175e+10,1.163250e+10,1.163250e+10,1.163826e+10,1.163826e+10,1.164536e+10,1.164787e+10,1.164787e+10,1.167086e+10,1.167735e+10,1.167977e+10,1.169591e+10,1.172136e+10,1.172136e+10,1.172136e+10,1.172136e+10,1.172217e+10,1.172217e+10,1.172510e+10,1.174274e+10,1.176191e+10,1.176523e+10,1.176644e+10,1.176644e+10,1.176785e+10,1.176785e+10,1.177409e+10,1.177684e+10,1.177824e+10,1.179254e+10,1.179333e+10,1.179770e+10,1.180017e+10
LUBELSKIE,0.00,0.00,0.00,0.00,0.00,1.631861e+07,7.483402e+07,8.039086e+07,1.221801e+08,1.386132e+08,1.621716e+08,2.746415e+08,3.604173e+08,3.817854e+08,3.939789e+08,8.537518e+08,8.720067e+08,1.168088e+09,1.168499e+09,1.168499e+09,1.187225e+09,2.022692e+09,2.615022e+09,2.621924e+09,4.299486e+09,4.548025e+09,4.787873e+09,5.053492e+09,5.207568e+09,5.207568e+09,5.221559e+09,5.232160e+09,5.246700e+09,5.254195e+09,5.266900e+09,5.277059e+09,1.228253e+10,1.391273e+10,1.391284e+10,1.391662e+10,1.392222e+10,1.393256e+10,1.394134e+10,1.395447e+10,1.397329e+10,1.398435e+10,1.399339e+10,1.399447e+10,1.401782e+10,1.403163e+10,1.404119e+10,1.406214e+10,1.406363e+10,1.406909e+10,1.408055e+10,1.410041e+10,1.410041e+10,1.410328e+10,1.413019e+10,1.413188e+10,1.413188e+10,1.413270e+10,1.413270e+10,1.414467e+10,1.414467e+10,1.414521e+10,1.415735e+10,1.415896e+10,1.415896e+10,1.416321e+10,1.417118e+10,1.417937e+10,1.418432e+10,1.418747e+10,1.418895e+10,1.419536e+10
LUBUSKIE,0.00,0.00,0.00,0.00,0.00,3.418706e+07,3.418706e+07,6.967835e+07,8.607041e+07,1.131594e+08,1.336899e+08,1.543975e+08,1.699440e+08,1.75799

In [75]:
#zapisujemy wyniki do pliku csv
df4.to_csv('/content/drive/MyDrive/Colab Notebooks/UE/umowy_PL_UE.csv')

# Wygenerowanie animowanego wykresu

Do utworzenia wykresu wykorzystujemy darmowe narzędzie dostępne na stronie https://flourish.studio/

Po utworzeniu konta:
1. Wybieramy "Bar chart race".
2. Ładujemy plik csv.
3. Dodajemy tytuł wykresu i ustawiamy opcje według potrzeb.




In [79]:
from IPython.display import HTML
s = """<div class="flourish-embed flourish-bar-chart-race" data-src="visualisation/6023241">
<script src="https://public.flourish.studio/resources/embed.js"></script></div>"""
h = HTML(s)
display(h)


Efekt końcowy jest dostępny również tu:
https://public.flourish.studio/visualisation/6023241/